# Find, Load, and Visualise Earth Observation Imagery

This notebook demonstrates how to find, load, and visualise Earth observation imagery using cloud native approaches, which work well on your desktop or in cloud environments!

The notebook demonstrates how to find images from the Datacube, load them, and visualise them using extensions to xarray provided by `odc-geo`.

## Set up

The first step is to set up the required Python libraries and local imports.

* `datacube` is used to access the Earth Search STAC catalog
* `numpy` is used to manipulate data
* `odc.ui` is used to make a location picker

In [ ]:
import numpy as np
from datacube import Datacube
from odc.geo.geom import point

The second step is to start a Dask client.

Dask supports local parallel processing and can help speed up computation times.

In [ ]:
from dask.distributed import Client

dask_client = Client()
dask_client

## Part 1: Find

### 1.1 Connecting to Datacube

In [ ]:
# Set up the Datacube
dc = Datacube()

### 1.2 Selecting an area to query

In [ ]:
lat, lon = xx, xx # Paste your lat, lon here
aoi = point(lon, lat, crs="epsg:4326")

polygon = aoi.buffer(0.1)
polygon.explore()

### 1.3 Set year and month to query

In [ ]:
# Set a start and end date
date_query = "2025-06"

### 1.4 Query for cloud cover

In [ ]:
cloud_cover_query = (0, 60)

### 1.5 Running the query to indentify Datasets

In [ ]:
product = "s2_l2a"

# Query with filtering for cloud cover
datasets = dc.find_datasets(
    product=product,
    lon=(bbox.left, bbox.right),
    lat=(bbox.bottom, bbox.top),
    time=date_query,
    cloud_cover=cloud_cover_query,
)

print(f"Found {len(datasets)} Datasets matching the query.")

## Part 2: Load

### 2.1 Using odc-stac to load identified items

This may take a few minutes

In [ ]:
# Load our filtered data
ds_filtered = dc.load(
    datasets=datasets,
    measurements=["red", "green", "blue"],
    dask_chunks={},
    resolution=20,
    group_by="solar_day",
    output_crs=datasets[0].crs,
    geopolygon=polygon,
).compute()

ds_filtered

### 2.2 Review loaded imagery

Identify which image you want to export and note the date.

In [ ]:
# To_array sets up a 3D array with the time dimension, which works directly
# with the plot function to make an RGB image
ds_filtered.to_array().plot.imshow(col="time", col_wrap=3, robust=True)

## Part 3: Visualise

### 3.1 Select best image

Update the `best_image_date` parameter to match the date you identified in the previous step.

In [ ]:
best_image_date = "2025-06-02"

best_image = ds_filtered.sel(time=best_image_date).squeeze()

best_image

### 3.2 View the selected image on an interactive map

In [ ]:
visualisation = best_image.odc.to_rgba()

visualisation.odc.explore()

### 3.3 Improve the image contrast

Calculate the values corresponding to the 1st and 99th percentiles. 
These can be used in the `to_rgba()` function to stretch the image.

In [ ]:
percentile_stretch = (1, 99)

rgb_array = best_image.to_array().values

stretch_vmin, stretch_vmax = np.nanpercentile(rgb_array, percentile_stretch)

Apply the percentile stretch values to the visualisation

In [ ]:
stretch_visualisation = best_image.odc.to_rgba(vmin=stretch_vmin, vmax=stretch_vmax)

stretch_visualisation.odc.explore()

### 3.4 Export to a cloud-optimised GeoTIFF

In [ ]:
stretch_visualisation.odc.write_cog("sentinel2_example.tif", overwrite=True)

## Part 4: Tidy up

In this section, we close the Dask client.
This prevents multiple clients being instantiated when using different notebooks.

In [ ]:
dask_client.close()